In [2]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
import statistics

%matplotlib inline

In [3]:
df = pd.read_csv('datasets/data.csv')
data = df[df.columns.difference(['Data'])]
data.shape

(92, 267)

In [4]:
x = data['ABCB4']
y = data['AALR3']

In [5]:
df = pd.DataFrame([[1, 2], [4, 5], [7, 8]],
     index=['cobra', 'viper', 'sidewinder'],
     columns=['max_speed', 'shield'])
df
 

,max_speed,shield
cobra,1,2
viper,4,5
sidewinder,7,8


In [8]:
df.loc['viper']

max_speed    4
shield       5
Name: viper, dtype: int64